### Загрузка всех изобраажений

In [7]:
# Структура словаря изображений 
# AR_IMAGES = {'name_category': [{'link':str,'images':[path]},...]}
#  'img_path_desc' = dataset/смартфон/1409765841

import os
import sqlite3
import pandas as pd
pp = os.getcwd()
PATH_DD = pp.split('group_mlada')[0] + 'group_mlada/'
# print(PATH_DD)
db_path = PATH_DD + 'dataset/ozon.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

/home/al/Projects_My/group_mlada/


In [8]:
query = """SELECT search_nm, link, img_path_desc
 FROM products;"""
strt_df = pd.read_sql_query(query, conn)

In [25]:
AR_IMAGES = {}
df_g_search_nm = strt_df.groupby('search_nm')
for key, _ in df_g_search_nm :
    df_category = df_g_search_nm .get_group(key)
    if key not in AR_IMAGES: 
        AR_IMAGES[key] = []
    llink = df_category['link'].to_list()
    for ll in llink[1:]:
        row = df_category.loc[df_category['link'] == ll, ['img_path_desc']]
        #print(list(row['img_path_desc']))
        if len(list(row['img_path_desc'])[0]) > 0:
            if list(row['img_path_desc'])[0].find('desc') >0 :
                dd = {'link': ll, 'images_dir': f"{PATH_DD}dataset/{key}/{list(row['img_path_desc'])[0].split('_')[1]}/"}
                AR_IMAGES[key].append(dd)
            # if list(row['img_path_desc'])[0].isdigit():
            #     print(list(row['img_path_desc']))
len(AR_IMAGES)
#'/home/al/Projects_My/group_mlada/dataset/смартфон/1623541758'

11

### Определение схожести между двумя объектами

In [12]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# import ssl

# ssl._create_default_https_context = ssl._create_stdlib_context

# Загрузка предобученной модели ResNet18
model = models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Удаление последнего слоя классификации
model.eval()

# Преобразования для изображений
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [13]:
def extract_features(image_path, model):
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = model(image).squeeze().numpy()
    return features

In [14]:
from scipy.spatial.distance import cosine

def are_images_similar(image_path1, image_path2, model, threshold=0.5):
    features1 = extract_features(image_path1, model)
    features2 = extract_features(image_path2, model)
    similarity = 1 - cosine(features1, features2)
    return similarity > threshold, similarity

In [17]:
# Пример использования
# image_path1 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810155852.webp'
# image_path2 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810156155.webp'
# image_path3 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н8/6836725625.webp'
# image_path4 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/2/7057584592.webp'
# image_path5 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/2/7057584629.webp'
# image_path6 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/9/6795482421.webp'

image_path0 = AR_IMAGES['кроссовки'][2]['images_dir'] + os.listdir(AR_IMAGES['кроссовки'][2]['images_dir'])[0]
print(image_path0)
image_path1 = AR_IMAGES['кроссовки'][2]['images_dir'] + os.listdir(AR_IMAGES['кроссовки'][2]['images_dir'])[3]
print(image_path1)
similar, similarity_score = are_images_similar(image_path0, image_path0, model)
print(f"Похожи ли изображения: {similar}, Схожесть: {similarity_score:.2f}")
similar, similarity_score = are_images_similar(image_path0, image_path1, model)
print(f"Похожи ли изображения: {similar}, Схожесть: {similarity_score:.2f}")

/home/al/Projects_My/group_mlada/dataset/кроссовки/175976781/image_7.jpg
/home/al/Projects_My/group_mlada/dataset/кроссовки/175976781/image_4.jpg
Похожи ли изображения: True, Схожесть: 1.00


### Определение объектов на изображении

In [26]:
import torchvision.transforms as T

# Загрузка модели для детекции объектов
object_detection_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
object_detection_model.eval()

# Преобразования для детекции
detection_transform = T.Compose([
    T.ToTensor(),
])

def detect_objects(image_path, model, threshold=0.5):
    image = Image.open(image_path)
    image_tensor = detection_transform(image)
    with torch.no_grad():
        predictions = model([image_tensor])[0]
    
    detected_objects = []
    for idx, score in enumerate(predictions['scores']):
        if score > threshold:
            label = predictions['labels'][idx].item()
            box = predictions['boxes'][idx].tolist()
            detected_objects.append((label, box, score.item()))
    
    return detected_objects


/home/al/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/al/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:

# Пример использования
image_path = AR_IMAGES['кроссовки'][2]['images_dir'] + os.listdir(AR_IMAGES['кроссовки'][2]['images_dir'])[0]
print(image_path)
detected_objects = detect_objects(image_path, object_detection_model)
print(f"Обнаруженные объекты: {detected_objects}")

/home/al/Projects_My/group_mlada/dataset/кроссовки/175976781/image_7.jpg
Обнаруженные объекты: [(31, [58.076900482177734, 585.86279296875, 636.2135620117188, 901.66015625], 0.503413200378418)]


### Определение схожести между списком объектов

In [29]:
image_paths = ['/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н9/6971608467.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч2/7021439607.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810155850.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/9/6795482421.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810156140.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч9/7061090493.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н3/6873617147.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н8/6122417752.webp']

In [31]:
from sklearn.decomposition import PCA

def cluster_images(image_paths, model, n_clusters=3):
    features = np.array([extract_features(image_path, model) for image_path in image_paths])
    
    # Для снижения размерности (опционально)
    min_samples = min(len(image_paths), features.shape[1])
    n_components = min(50, min_samples - 1)  # Убедиться, что n_components допустимо
    
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    
    # Кластеризация
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(reduced_features)
    
    return clusters


In [38]:

image_paths = AR_IMAGES['кроссовки'][2]['images_dir']
ll_image_paths = os.listdir(AR_IMAGES['кроссовки'][2]['images_dir'])
print(image_paths)
print(ll_image_paths)
for i1 in ll_image_paths:
    for i2 in ll_image_paths:
        clusters = cluster_images([image_paths + i1, image_paths + i2], model, n_clusters=2)
        print(f"Кластеры изображений: {clusters}")

/home/al/Projects_My/group_mlada/dataset/кроссовки/175976781/
['image_7.jpg', 'image_5.jpg', 'image_0.jpg', 'image_4.jpg', 'image_2.jpg', 'image_3.jpg', 'image_6.jpg', 'image_1.jpg']
Кластеры изображений: [0 0]


/home/al/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Кластеры изображений: [0 1]
Кластеры изображений: [1 0]


/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Кластеры изображений: [0 1]
Кластеры изображений: [0 1]


...

/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Кластеры изображений: [1 0]
Кластеры изображений: [1 0]


/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Кластеры изображений: [1 0]
Кластеры изображений: [0 0]


/home/al/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


In [39]:
def cluster_images_without_pca(image_paths, model, n_clusters=3):
    features = np.array([extract_features(image_path, model) for image_path in image_paths])
    
    # Кластеризация
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(features)
    
    return clusters

clusters = cluster_images_without_pca(image_paths, model, n_clusters=3)
print(f"Кластеры изображений: {clusters}")

IsADirectoryError: [Errno 21] Is a directory: '/'